In [24]:
%pylab inline
import gc
import galsim
import fpfsBase
import imgSimutil
import numpy as np
import astropy.io.fits as pyfits
import numpy.lib.recfunctions as rfn

pltDir='plot/errors/'
vdir='outcome-var7em3'
def measure(galImgAll):
    imgList=[galImgAll[i//nx*ngrid+beg:i//nx*ngrid+end,i%nx*ngrid+beg:i%nx*ngrid+end] for i in range(ngal)]
    a=fpTask.measure(imgList)
    del imgList
    gc.collect()
    return a

Populating the interactive namespace from numpy and matplotlib


In [7]:
cd wrkDir/FPFS2/sim

/lustre/work/xiangchong.li/work/FPFS2/sim


In [8]:
mmsN0=[]
mmsN1=[]
mmsN2=[]
for igroup in range(16):
    for ii in range(50):
        ifield=igroup*250+ii
        fname=os.path.join(vdir,'fpfs-rcut16-psf60-%d/src%04d-g1-0000.fits' %(igroup,ifield))
        mm=pyfits.getdata(fname)
        mmsN0.append(mm)
        del mm,fname
        
        fname=os.path.join(vdir,'fpfs-rcut16-psf60-%d/src%04d-g1-1111.fits' %(igroup,ifield))
        mm=pyfits.getdata(fname)
        mmsN1.append(mm)
        del mm,fname
        gc.collect()
        
        fname=os.path.join(vdir,'fpfs-rcut16-psf60-%d/src%04d-g1-2222.fits' %(igroup,ifield))
        mm=pyfits.getdata(fname)
        mmsN2.append(mm)
        del mm,fname
        gc.collect()
        
mmsN0=rfn.stack_arrays(mmsN0,usemask=False)
mmsN1=rfn.stack_arrays(mmsN1,usemask=False)
mmsN2=rfn.stack_arrays(mmsN2,usemask=False)

In [14]:
C=2000.
bN0=fpfsBase.fpfsM2E(mmsN0,C,rev=True)
cN0=fpfsBase.fpfsM2Err(mmsN0,C)

bN1=fpfsBase.fpfsM2E(mmsN1,C,rev=True)
cN1=fpfsBase.fpfsM2Err(mmsN1,C)

bN2=fpfsBase.fpfsM2E(mmsN2,C,rev=True)
cN2=fpfsBase.fpfsM2Err(mmsN2,C)

In [13]:
def main(bN,cN,pend='0000',cc=2000):
    cutList=np.linspace(0.,0.4,8)
    deList=[]

    nx=5000
    ny=5000

    hist,ebins,sbins=np.histogram2d(bN['fpfs_e1'],bN['fpfs_RE'],range=((-0.6,0.6),(-0.2,1.1)),bins=(5000,5000))
    ePix=ebins[1]-ebins[0]
    sPix=sbins[1]-sbins[0]
    sCenter,eCenter=np.meshgrid((sbins[:-1]+sbins[1:])/2.,(ebins[:-1]+ebins[1:])/2.)
    sigma_x = np.sqrt(np.average(cN['fpfs_s0Err']))/sPix
    sigma_y = np.sqrt(np.average(cN['fpfs_e1Err']))/ePix
    rho=np.average(cN['fpfs_e1s0Cov'])/sigma_x/sigma_y/sPix/ePix
    print(sigma_x,sigma_y,rho)
    x=np.array([np.fft.fftfreq(nx)*nx])
    y=np.fft.fftfreq(ny)*ny
    y=np.transpose(np.array([y]))
    psf  = 1./(2*np.pi*sigma_x*sigma_y)*np.exp(-(x**2/(2*sigma_x**2)-2.*rho*x*y/sigma_x/sigma_y/2. + y**2/(2*sigma_y**2)))
    if False:
        test=np.zeros(psf.shape)
        test[150,150]=1.
        test2=np.real(np.fft.ifft2(np.fft.fft2(test)*np.fft.fft2(psf)))
        plt.imshow(test2[120:180,120:180],origin='lower')
    hist[:,:10]=0.
    hist[:,-10:]=0.
    histC=np.real(np.fft.ifft2(np.fft.fft2(hist)*np.fft.fft2(psf)))
    for clow in cutList:
        msk=sCenter>clow
        de=np.sum(eCenter[msk]*histC[msk])/np.sum(histC[msk])-np.sum(eCenter[msk]*hist[msk])/np.sum(hist[msk])
        deList.append(de)
    deList=np.array(deList)
    out={}
    out.update({'cutList':cutList})
    out.update({'deList':deList})
    np.save(os.path.join(pltDir,'deList_C%d_%s.npy' %(int(cc),pend)),out)
    return

In [14]:
main(bN0,cN0,'0000',C)
main(bN1,cN1,'1111',C)
main(bN2,cN2,'2222',C)

36.618659025424314 112.18598571729628 0.012923583258824756


In [ ]:
if False:
    print(np.load(os.path.join(pltDir,'deList_C4000_0000.npy'),allow_pickle=True).item()['deList'])
    print(np.load(os.path.join(pltDir,'deList_C4000_1111.npy'),allow_pickle=True).item()['deList'])
    print(np.load(os.path.join(pltDir,'deList_C4000_2222.npy'),allow_pickle=True).item()['deList'])

In [3]:
cosmo252=   imgSimutil.cosmoHSTGal('252')
cosmo252.readHSTsample()
hscCat  =   cosmo252.catused

In [25]:
def main2(bN,cN,zAll,pend='0000',cc=2000):
    clow=0.15
    cutList=np.linspace(0.,0.4,8)
    deList=[]
    nx=5000
    ny=5000
    for ic in range(3):
        zlow = 0.6*(ic)
        zhigh= 0.6*(1+ic)
        maskZ= (zAll>zlow)&(zAll<=zhigh)
        hist,ebins,sbins=np.histogram2d(bN[maskZ]['fpfs_e1'],\
                                        bN[maskZ]['fpfs_RE'],\
                                        range=((-0.6,0.6),(-0.2,1.1)),\
                                        bins=(5000,5000))
        ePix=ebins[1]-ebins[0]
        sPix=sbins[1]-sbins[0]
        sCenter,eCenter=np.meshgrid((sbins[:-1]+sbins[1:])/2.,(ebins[:-1]+ebins[1:])/2.)
        msk=sCenter>clow
        sigma_x = np.sqrt(np.average(cN['fpfs_s0Err']))/sPix
        sigma_y = np.sqrt(np.average(cN['fpfs_e1Err']))/ePix
        rho=np.average(cN['fpfs_e1s0Cov'])/sigma_x/sigma_y/sPix/ePix
        print(sigma_x,sigma_y,rho)
        x=np.array([np.fft.fftfreq(nx)*nx])
        y=np.fft.fftfreq(ny)*ny
        y=np.transpose(np.array([y]))
        psf  = 1./(2*np.pi*sigma_x*sigma_y)*np.exp(-(x**2/(2*sigma_x**2)-2.*rho*x*y/sigma_x/sigma_y/2. + y**2/(2*sigma_y**2)))
        hist[:,:10]=0.
        hist[:,-10:]=0.
        histC=np.real(np.fft.ifft2(np.fft.fft2(hist)*np.fft.fft2(psf)))
        de=np.sum(eCenter[msk]*histC[msk])/np.sum(histC[msk])-np.sum(eCenter[msk]*hist[msk])/np.sum(hist[msk])
        deList.append(de)
        del hist,histC,psf
        gc.collect()
    deList=np.array(deList)
    out={}
    out.update({'cutList':cutList})
    out.update({'deList':deList})
    np.save(os.path.join(pltDir,'deList_C%d_%s_low15_redshift3.npy' %(int(cc),pend)),out)
    return

In [11]:
zAll=[]
for igroup in range(16):
    ztmp=hscCat[10000*(igroup%8):10000*(igroup%8+1)]['zphot']
    for ii in range(50):
        zAll.append(ztmp)
    del ztmp
    gc.collect()
zAll=rfn.stack_arrays(zAll,usemask=False)

In [26]:
main2(bN0,cN0,zAll,'0000',C)
main2(bN1,cN1,zAll,'1111',C)
main2(bN2,cN2,zAll,'2222',C)

36.618659025424314 112.18598571729628 0.012923583258824756
36.618659025424314 112.18598571729628 0.012923583258824756
36.618659025424314 112.18598571729628 0.012923583258824756
36.61613239830711 112.18055530739059 0.013212824380568292
36.61613239830711 112.18055530739059 0.013212824380568292
36.61613239830711 112.18055530739059 0.013212824380568292
36.617855618750674 112.17127862388719 0.013501616151189114
36.617855618750674 112.17127862388719 0.013501616151189114
36.617855618750674 112.17127862388719 0.013501616151189114


In [29]:
print(np.load(os.path.join(pltDir,'deList_C2000_0000_low15_redshift3.npy'),allow_pickle=True).item()['deList'])
print(np.load(os.path.join(pltDir,'deList_C2000_1111_low15_redshift3.npy'),allow_pickle=True).item()['deList'])
print(np.load(os.path.join(pltDir,'deList_C2000_2222_low15_redshift3.npy'),allow_pickle=True).item()['deList'])

[-1.59336373e-06  3.17897298e-06 -7.95844965e-07]
[8.73466113e-06 1.30175002e-05 1.06412064e-05]
[1.65516745e-05 2.29961174e-05 3.93911328e-05]
